# Getting started with MillionTrees datasets

In [1]:
import os
import sys

if os.path.basename(os.getcwd()) == 'examples':
    sys.path.append("../")
    
import milliontrees
from torchvision import transforms

# List available datasets
print(milliontrees.benchmark_datasets)

['TreePoints', 'TreeBoxes', 'TreePolygons']


The general workflow is to 
1. Select and optionally download a dataset
2. Load the train and test splits from the dataset
3. Create the dataloader, with optional additional transforms, for how to preprocess images, and optionally augment, input images and metadata
4. Use these dataloaders to train models in native pytorch or pytorch lightning

### Select and optionally download a dataset

In [2]:
# Load the box dataset
from milliontrees import get_dataset
dataset = get_dataset("TreeBoxes", root_dir="/orange/ewhite/DeepForest/MillionTrees/")

/blue/ewhite/b.weinstein/miniconda3/envs/MillionTrees/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/blue/ewhite/b.weinstein/miniconda3/envs/MillionTrees/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


### Load the train and test splits from the dataset

Datasets are split into groups of images based on task. For example, 'train' versus 'test' or 'zero_shot_train' and 'zero_shot_test'. The get_subset function has a 'frac' argument to subsample the data during rapid testing and debugging.

In [3]:
from milliontrees.common.data_loaders import get_train_loader

train_dataset = dataset.get_subset("train")

# View the first image in the dataset
metadata, image, targets = train_dataset[0]
print(f"Metadata length: {len(metadata)}")
print(f"Image shape: {image.shape}, Image type: {type(image)}")
print(f"Targets keys: {targets.keys()}, Label type: {type(targets)}")

Metadata length: 2
Image shape: torch.Size([3, 448, 448]), Image type: <class 'torch.Tensor'>
Targets keys: dict_keys(['y', 'labels']), Label type: <class 'dict'>


### Create dataloader

In [4]:
train_loader = get_train_loader("standard", train_dataset, batch_size=2)

# Show one batch of the loader
for metadata, image, targets in train_loader:
    print("Targets is a list of dictionaries with the following keys: ", targets[0].keys())
    print(f"Image shape: {image.shape}, Image type: {type(image)}")
    print(f"Annotation shape of the first image: {targets[0]['y'].shape}")
    break  # Just show the first batch

Targets is a list of dictionaries with the following keys:  dict_keys(['y', 'labels'])
Image shape: torch.Size([2, 3, 448, 448]), Image type: <class 'torch.Tensor'>
Annotation shape of the first image: torch.Size([10, 4])


## Evaluation loader

In [ ]:
from milliontrees.common.data_loaders import get_eval_loader
import torch

test_dataset = dataset.get_subset("train")
test_loader = get_eval_loader("standard", test_dataset, batch_size=16)

# Show one batch of the loader
all_y_pred = []
all_y_true = []
all_metadata = []

def predictor(images):
    return [{'y': torch.tensor([[30, 70, 35, 75]]),
             'label': torch.tensor([0]),
             'score': torch.tensor([0.54])} for _ in range(images.shape[0])]

# For the sake of this example, we will make up some predictions to show format
for metadata, images, targets in test_loader:
    all_metadata.append(metadata)
    all_y_true.append(targets)
    all_y_pred.append(predictor(images))


# Evaluate
dataset.eval(all_y_pred, all_y_true, all_metadata)

/blue/ewhite/b.weinstein/miniconda3/envs/MillionTrees/lib/python3.10/site-packages/albumentations/core/bbox_utils.py:468: RuntimeWarning: invalid value encountered in divide
  & (clipped_box_areas / denormalized_box_areas >= min_visibility - EPSILON)


KeyError: 'score'